In [1]:
cd ../

d:\OneDrive\Documents\Monash Research Project T2 2022\HazardousWaste


In [2]:
from Lib.WebScrapping.CorpusScraper import *
from Lib.TextAnalytics.Methods import *
import sys
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
webpages = get_tag_corpus(["metal", "energy", "hazard", "liquid", "battery"], 1)

2022-09-18 22:32:42 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-09-18 22:32:42 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2022-09-18 22:32:42 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Edg/105.0.1343.27'}
2022-09-18 22:32:42 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-09-18 22:32:42 [scrapy.extensions.telnet] INFO: Telnet Password: 972cedda7208d4f6
2022-09-18 22:32:42 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-09-18 22:32:42 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scr

Running Google Search at https://www.google.com.au/search?q=metal%20energy%20hazard%20liquid%20battery&start=0


2022-09-18 22:32:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://undecidedmf.com/episodes/hot-energy-storage-liquid-metal-battery-explained/> (referer: None)
2022-09-18 22:32:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://undecidedmf.com/episodes/hot-energy-storage-liquid-metal-battery-explained/>
{'content': ['https://undecidedmf.com/episodes/hot-energy-storage-liquid-metal-battery-explained/', 'Html', 'Hot Energy Storage? Liquid Metal Battery Explained - Undecided with Matt Ferrell', "\nSearch   Menu\n\nHome\n\nRenewable Energy\n\nEnergy Production\n\nEnergy Storage\n\nAll Renewable Posts\n\nSustainable Tech\n\nClean Transport\n\nElectric Vehicles\n\nAll Clean Transportation Posts\n\nReviews\n\nElectrify Everything\n\nSmart Home\n\nAll Reviews\n\nVideos\n\nPodcasts\n\nShop\n\nSearch\n\nEnergy Storage\n\nHot Energy Storage? Liquid Metal Battery Explained\n\nBy Matt Ferrell\n\nJuly 13, 2021\n\nShare\n\nTweet\n\n0\n\nSolar and wind power have proven themselves to be

Running keyword collection at https://undecidedmf.com/episodes/hot-energy-storage-liquid-metal-battery-explained/


2022-09-18 22:32:45 [scrapy.core.engine] INFO: Spider closed (finished)
2022-09-18 22:32:45 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-09-18 22:32:45 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2022-09-18 22:32:45 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Edg/105.0.1343.27'}
2022-09-18 22:32:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-09-18 22:32:45 [scrapy.extensions.telnet] INFO: Telnet Password: c2c17a5b79752189
2022-09-18 22:32:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-09-18 

...

2022-09-18 22:32:48 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-09-18 22:32:48 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2022-09-18 22:32:48 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Edg/105.0.1343.27'}
2022-09-18 22:32:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-09-18 22:32:48 [scrapy.extensions.telnet] INFO: Telnet Password: 3a1f1874be3b603d
2022-09-18 22:32:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-09-18 22:32:48 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scr

Running keyword collection at https://www.energy-storage.news/ambri-gets-ul-1973-safety-certification-for-liquid-metal-battery-storage-tech/


2022-09-18 22:32:50 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.energy-storage.news/ambri-gets-ul-1973-safety-certification-for-liquid-metal-battery-storage-tech/>
{'content': ['https://www.energy-storage.news/ambri-gets-ul-1973-safety-certification-for-liquid-metal-battery-storage-tech/', 'Html', 'Ambri gets UL 1973 safety certification for liquid metal battery storage tech - Energy Storage News', "\nSolar Media\n\nEvents\n\nPV Tech\n\nSolar Power Portal\n\nCurrent News\n\nTwitter\n\nLinkedIn\n\nYouTube\n\nFacebook\n\nFeed\n\nAdvertising\n\nContact\n\nResults\n\nSee all results\n\nHome\n\nNews\n\nCommercial\n\nResidential\n\nGrid Scale\n\nOff Grid\n\nMarket Watch\n\nConnected Tech\n\nResources\n\nEvents &amp; Webinars\n\nUpcoming Events\n\nUpcoming Webinars\n\nOn-demand Webinars\n\nFeatures\n\ngoogletag.cmd.push(function() { googletag.display('div-gpt-ad-1619444166567-0'); });\n\nNews\n\nAmbri gets UL 1973 safety certification for liquid metal battery storage tech\n\nBy

Running keyword collection at https://www.worksafe.qld.gov.au/safety-and-prevention/hazards/electricity/hazardous-electrical-environments/battery-energy-storage-systems


2022-09-18 22:32:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.worksafe.qld.gov.au/safety-and-prevention/hazards/electricity/hazardous-electrical-environments/battery-energy-storage-systems>
{'content': ['https://www.worksafe.qld.gov.au/safety-and-prevention/hazards/electricity/hazardous-electrical-environments/battery-energy-storage-systems', 'Html', 'Battery energy storage systems (BESS) | WorkSafe.qld.gov.au', '\nSkip to content\n\nWorkSafe .qld.gov.au\n\nMenu\n\nAbout\n\nContact\n\nServices\n\nNews &amp; events\n\n1300 362 128\n\nSearch\n\nLogin\n\nWorker Assist (Opens in an external website)\n\nWorkCover Connect (Opens in an external website)\n\nProvider Connect (Opens in an external website)\n\nHigh risk work assessor portal (Opens in an external website)\n\nHigh risk work applicant/licencee services (Opens in an external website)\n\nElectrical licensing office course package and online assessment (Opens in an external website)\n\nOrganisational systems benchmarki

In [ ]:
from Lib.PreProcessing.General import *
corpus = [remove_stop_words(webpage.textual_content, 'Resources/stop-words.txt') for webpage in webpages]

In [ ]:
result = get_tfidf(corpus)

In [ ]:
features = result[1].get_feature_names_out()

In [ ]:
non_zero_results = [csr_matrix.nonzero() for csr_matrix in result[0]]

In [ ]:
non_zero_result_importance = list()
for i, non_zero_result in enumerate(non_zero_results):
    temp = list()
    for (row, col) in zip(*non_zero_result):
        temp.append(((row, col), result[0][i][row, col]))
    temp.sort(key=lambda a: a[1], reverse=True)
    non_zero_result_importance.append(temp)

In [ ]:
top_10 = np.empty((len(features), len(webpages)), dtype=float)
for idx, non_zero_result in enumerate(non_zero_result_importance):
    for ((row, col), importance) in non_zero_result:
        top_10[col, idx] = importance
top_10 = np.nan_to_num(top_10, posinf=1, neginf=0)

In [ ]:
df = pd.DataFrame(top_10, index=features, columns=[webpage.title for webpage in webpages])

In [ ]:
for webpage in webpages:
    print(webpage.title, webpage.url)

A Decade In Development, Liquid-Metal Batteries By Ambri May Soon Change The Energy Storage Game https://www.forbes.com/sites/davidblackmon/2021/09/02/bill-gates-backed-startup-might-change-the-renewable-energy-storage-game/
Hot Energy Storage? Liquid Metal Battery Explained - Undecided with Matt Ferrell https://undecidedmf.com/episodes/hot-energy-storage-liquid-metal-battery-explained/
Liquid Metal Battery - an overview | ScienceDirect Topics https://www.sciencedirect.com/topics/engineering/liquid-metal-battery
Ambri gets UL 1973 safety certification for liquid metal battery storage tech - Energy Storage News https://www.energy-storage.news/ambri-gets-ul-1973-safety-certification-for-liquid-metal-battery-storage-tech/
Battery energy storage systems (BESS) | WorkSafe.qld.gov.au https://www.worksafe.qld.gov.au/safety-and-prevention/hazards/electricity/hazardous-electrical-environments/battery-energy-storage-systems


In [ ]:
threshold = 0.2
top_keywords = df.loc[(df > threshold).any(axis=1), (df > threshold).any()]
top_keywords
#top_keywords.style.background_gradient()

9.904179070203794e+218